In [1]:
from delta import *
from pyspark.sql import SparkSession

builder = SparkSession.builder.appName('delta-tutorial').config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension').config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/06/14 12:09:37 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.237 instead (on interface wlp5s0)
23/06/14 12:09:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aadi/.ivy2/cache
The jars for the packages stored in: /home/aadi/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0b38177c-336b-4cab-a721-7b0f9f2726d1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 94ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   | 

23/06/14 12:09:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/14 12:09:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [17]:
from pandas import DataFrame, Series, date_range

from numpy import random



sdf.dtypes

/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


[('id', 'string'),
 ('year_month', 'timestamp'),
 ('num1', 'double'),
 ('num2', 'double'),
 ('num3', 'double')]

In [46]:
import time
from pyspark.sql import functions as F
from pyspark.sql.window import Window

## with looping
A = 360
B = 480
sdf = spark.createDataFrame(DataFrame({
  'id': ['a'] * A + ['b'] * B, 
  'year_month': list(date_range(start='2021-01-01', periods=A, freq='m')) + list(date_range(start='2020-01-01', periods=B, freq='m')),
  'num1': random.normal(loc=1000, scale=200, size=A + B),
  'num2': random.normal(loc=1000, scale=200, size= A + B),
  'num3': random.normal(loc=1000, scale=200, size=A + B),
}))

window = (
    Window
    .partitionBy('id')
    .orderBy('year_month')
    .rowsBetween(-2, 0)
)

for w in [3, 6, 12]:
  window = (
      Window
      .partitionBy('id')
      .orderBy('year_month')
      .rowsBetween(-w, 0)
  )
  for col in ['num1', 'num2','num3']:
    sdf = sdf.withColumn('num1_window', F.avg(col).over(window))

start = time.perf_counter()
sdf.collect()

print(time.perf_counter() - start)

0.1073195249773562


In [47]:
# without looping
A = 360
B = 480
sdf = spark.createDataFrame(DataFrame({
  'id': ['a'] * A + ['b'] * B, 
  'year_month': list(date_range(start='2021-01-01', periods=A, freq='m')) + list(date_range(start='2020-01-01', periods=B, freq='m')),
  'num1': random.normal(loc=1000, scale=200, size=A + B),
  'num2': random.normal(loc=1000, scale=200, size= A + B),
  'num3': random.normal(loc=1000, scale=200, size=A + B),
}))


window = (
    Window
    .partitionBy('id')
    .orderBy('year_month')
    .rowsBetween(-2, 0)
)


sdf = sdf.select(["*"] + [F.avg(f"{col}").over(
  Window.partitionBy('id')
  .orderBy('year_month')
  .rowsBetween(-w+1, 0)
).alias(f'{col}_window') for col in ['num1', 'num2', 'num3'] for w in [3, 6, 9]]) # type: ignore



start = time.perf_counter()
sdf.collect()
print(time.perf_counter() - start)

0.12233384099090472


In [43]:
sdf.show()

+---+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+
| id|         year_month|              num1|              num2|              num3|       num1_window|       num2_window|       num3_window|
+---+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  a|2021-01-31 00:00:00| 838.0094799034778|1070.5663099934297|1155.3446824718862| 838.0094799034778|1070.5663099934297|1155.3446824718862|
|  a|2021-02-28 00:00:00| 902.7514364697699| 721.9839584217593|1162.8678481010104| 870.3804581866239| 896.2751342075945|1159.1062652864484|
|  a|2021-03-31 00:00:00| 941.4787028865572|1416.0157835140012| 974.5254693643411| 894.0798730866017|1069.5220173097302|1097.5793333124127|
|  a|2021-04-30 00:00:00| 992.1368768305997| 1136.601558323544|1142.9520545473276|  945.455672062309|1091.5337667531014|1093.4484573375596|
|  a|2021-05-31 00:0